# Power

This notebook contains code to assess power and determine the optimal $n\_\{variable\}$ in an experiment. 

## Recommendations
1. Please ensure that you have install `wiscs`. 
2. Read the descriptions in the markdown cells carefully.

>This is a low-compute demo. It's scaled up in separate code for HPC. 

## Imports

In [8]:
# if you wish to run the simulation approach, please install the following packages
!pip install git+https://github.com/w-decker/wiscs.git --quiet # REQUIRED FOR THIS NOTEBOOK
!pip install git+https://github.com/w-decker/rinterface.git --quiet # REQUIRED FOR THIS NOTEBOOK

In [2]:
import rinterface.rinterface as R
from rinterface.utils import to_r

from src.power import grid
from src.utils import fmt_script

import wiscs
from wiscs.utils import make_tasks
from wiscs.simulate import DataGenerator
from wiscs.formula import Formula

import numpy as np
from tqdm import tqdm

## Description of simulation based approach

A simulation based approach to calculating $n\_\{variable\}$ for an experiment is simple, but it involves a few steps.

1. First, a combination, $C$, of all possible values of parameters is generated (See sample slice below).

|   | n_items | n_participants | 
|---|---------|----------------|
| 0 | 10      | 10             | 
| 1 | 10      | 20             | 
| 2 | 10      | 30             | 
| 3 | 10      | 40             |
| 4 | 10      | 50             | 
| 5 | 20      | 10             | 
| 6 | 20      | 20             | 
| 7 | 20      | 30             | 
| 8 | 20      | 40             | 
| 9 | 20      | 50             | 

2. Then for $k$ iterations, a dataset is randomly generated for each row in $C$.

3. Two linear modeld are run with the generated data. The first being something like `Y ~ FE1 + FE 2 + (1 + FE2 | subject)` and the second `Y ~ FE1 + FE + (FE1 : FE2) + (1 + FE2 | subject)`. If the AIC of model one is lower or there is no statistical difference between the two models, then a counter, $T$ (which is initialized with $0$) is updated with $+1$.

7. Desired power is reached when $\frac{\text{sum}(T)}{k} = \text{desired power}$. 

See [here](https://cran.r-project.org/web/packages/SimEngine/vignettes/example_1.html#:~:text=The%20basic%20idea%20is%20that,this%20is%20your%20estimated%20power.) for more details.

## Let's generate some data

See [generate_data.ipynb](/notebooks/generate_data.ipynb) for details regarding how data are generated.

In [7]:
# If you wish to generate data based on parameters you wish to define directly in the code, modify + run this cell
n_question = 6
n_item = 30
n_subject = 1000
task = make_tasks(low=100, high=200, n=n_question, seed=2025)
re_formula = Formula("(1 + question + modality | subject) + (1 + question + modality | item)")
question_sd = [10, 12, 15, 18, 11] # must be n_q - 1
params = {'word.perceptual': 100, 'image.perceptual': 95, 'word.conceptual': 100, 'image.conceptual': 100, 'word.task': task, 'image.task': task,
        # noise parameters     
        'sd.item': 30,     'sd.question': question_sd[:n_question-1],    'sd.subject': 20,       "sd.modality": 10, "sd.error": 50, "sd.re_formula": str(re_formula),
        # correlations among random effects    
        "corr.subject": np.eye(n_question + 1), 'corr.item':np.eye(n_question + 1),
        # design parameters
        'n.subject': n_subject, 'n.question': n_question, 'n.item': n_item
}
wiscs.set_params(params)

# generate data
DG = DataGenerator()
df = DG.fit_transform(seed=2025).to_pandas().to_csv(f'data_{n_item}_{n_question}.csv', index=False)

Params set successfully


In [ ]:
{'sd.question': question_sd[:n_question-1], 'corr.subject': np.eye(n_question + 1), 'corr.item':np.eye(n_question + 1), 'n.question': n_question, 'n.item': n_item, 'n.subject': n_subject}

In [38]:
R(fmt_script(shared_re=re_formula, df=df))

Warning message:
In commonArgs(par, fn, control, environment()) :
  maxfun < 10 * length(par)^2 is not recommended.



 SHARED model summary
Linear mixed model fit by maximum likelihood . t-tests use Satterthwaite's
  method [lmerModLmerTest]
Formula: rt ~ modality + question + (1 + question + modality | subject) +  
    (1 + question + modality | item)
   Data: df
Control: lmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 10000))

      AIC       BIC    logLik  deviance  df.resid 
 322204.3  322611.5 -161053.2  322106.3     29951 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.9269 -0.6587  0.0030  0.6682  3.7449 

Random effects:
 Groups   Name        Variance Std.Dev. Corr                         
 subject  (Intercept)  427.83  20.684                                
          question1    129.68  11.388   -0.16                        
          question2    213.78  14.621   -0.22  0.30                  
          question3    237.47  15.410    0.00  0.02  0.27            
          question4    297.88  17.259   -0.11  0.12  0.15  0.16      
          modality1    104.99  10

Warning message:
In commonArgs(par, fn, control, environment()) :
  maxfun < 10 * length(par)^2 is not recommended.


Below are some variables

In [3]:
n_iter = 10 # how many interations to do?
desired_power = 0.8
p_threshold = 0.05

In [11]:
# If you wish to define a possible range of n_{variable}, modify + run this cell

#################################################################
n_subjects_range = np.arange(100, 500, 50) # range of subjects to test
n_items_range = np.arange(10, 50, 10) # range of items to test
n_questions_range = np.arange(2, 5, 1) # range of questions to test
#################################################################

combinations = grid(subjects=n_subjects_range, items=n_items_range, questions=n_questions_range) # get all combinatinos

In [12]:
combinations

array([[100,  10,   2],
       [100,  20,   2],
       [100,  30,   2],
       [100,  40,   2],
       [150,  10,   2],
       [150,  20,   2],
       [150,  30,   2],
       [150,  40,   2],
       [200,  10,   2],
       [200,  20,   2],
       [200,  30,   2],
       [200,  40,   2],
       [250,  10,   2],
       [250,  20,   2],
       [250,  30,   2],
       [250,  40,   2],
       [300,  10,   2],
       [300,  20,   2],
       [300,  30,   2],
       [300,  40,   2],
       [350,  10,   2],
       [350,  20,   2],
       [350,  30,   2],
       [350,  40,   2],
       [400,  10,   2],
       [400,  20,   2],
       [400,  30,   2],
       [400,  40,   2],
       [450,  10,   2],
       [450,  20,   2],
       [450,  30,   2],
       [450,  40,   2],
       [100,  10,   3],
       [100,  20,   3],
       [100,  30,   3],
       [100,  40,   3],
       [150,  10,   3],
       [150,  20,   3],
       [150,  30,   3],
       [150,  40,   3],
       [200,  10,   3],
       [200,  20

## Simulation based approach



`rinterface.rinterface()`, a (somewhat clumsy but convienient) way to interface with R in Python is how we will run linear models. Below is is the R code we will run

In [13]:
# code for model eval in R
def code(df, p_threshold):
    return (f"""
    suppressMessages(library(lme4))
    suppressMessages(library(psych))
    suppressMessages(library(dplyr))
    suppressMessages(library(lmerTest))

    # import data from Python
    df <- {to_r(df)}

    # factorize + treatment coding
    df$question <- as.factor(df$question)
    df$subject <- as.factor(df$subject)
    df$item <- as.factor(df$item)
    df$modality <- factor(df$modality, levels = c("word", "image"))
    contrasts(df$modality) <- c(-0.5, 0.5)

    #  set reference levels
    df$question <- relevel(df$question, ref = "0")
    df$item <- relevel(df$item, ref = "0")

    # load data
    df <- {to_r(df)}

    # model
    control <- lmerControl(optimizer = "bobyqa")
    shared <- lmer(rt ~ modality + question + (1 + question + modality | subject) + (1 + question + modality| item), data = df, REML = FALSE, control = control) # nolint
    separate <- lmer(rt ~ modality * question + (1 + question | subject) + (1 | item), data = df, REML = FALSE, control = control) # nolint

    # compare
    aicvalues <- c("Shared" = AIC(shared), "Separate" = AIC(separate))
    p_value <- anova(shared, separate, test="Chisq")$`Pr(>Chisq)`[2]

    # @grab{str}
    success <- ifelse(p_value > {p_threshold}, 1, 0)
    """)

Below is the loop

In [14]:
# iter = tqdm(combinations) # instantiate iter obj
iter = tqdm(n_subjects_range) # instantiate iter obj

results = {}  # Dictionary to store results

for j, _ in enumerate(iter):
    success = []
    power = 0
    for i in range(n_iter):

        # update data

        # Run the R model and determine winner
        winner = R(code(df, p_threshold), grab=True)
        success.append(1 if winner == 'Shared' else 0)

        # Calculate current power
        power = np.sum(success) / (i + 1)
        
        # update tqdm
        iter.set_postfix({
            "Power": round(power, 3), 
            "Iteration": i + 1, 
            "# Winners": np.sum(success), 
            "# Losers": success.count(0)
        })

        # Check power / if power is possible
        if success.count(0) == n_iter - (0.8 * n_iter) + 1: 
             break
        elif power >= desired_power:
            results[j] = power
            iter.set_postfix({"Power": round(power, 3), "Status": "Stopping early"})
            break  # Break out of the inner loop once desired power is reached


    if power >= desired_power:
        break


 38%|███▊      | 3/8 [04:50<08:03, 96.70s/it, Power=0, Iteration=2, # Winners=0, # Losers=2]


KeyboardInterrupt: 